In [1]:
!git clone https://github.com/TheMrGhostman/Semi-supervised-Learning-with-VAE.git

Cloning into 'Semi-supervised-Learning-with-VAE'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 780 (delta 49), reused 78 (delta 34), pack-reused 678
Receiving objects: 100% (780/780), 321.04 MiB | 33.43 MiB/s, done.
Resolving deltas: 100% (280/280), done.
Checking out files: 100% (496/496), done.


In [0]:
import os 
os.chdir("Semi-supervised-Learning-with-VAE/")

In [0]:
import numpy as np
import pandas as pd
import os 
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from utils.inference import Trainer
import utils.models as m
import utils.datasets as d
from utils.layers import *

# Data

In [0]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [0]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666)

In [0]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=666)


In [0]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_valid, y_valid)
test = d.H_alphaSequences(X_test, y_test)

In [0]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [0]:
VAE = torch.load("models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300.pt")

# Freeze and Unfreeze

In [0]:
class RNN_Encoder_Classifier(nn.Module):
    def __init__(self, model, clf):
        super(RNN_Encoder_Classifier, self).__init__()
        self.model = model
        self.clf = clf
        self.frozen_encoder = False

    def freeze_encoder(self):
        for param in self.model.parameters():
            param.requires_grad = False
        self.frozen_encoder = True
        print("Encoder frozen.")

    def unfreeze_encoder(self):
        for param in self.model.parameters():
            param.requires_grad = True
        self.frozen_encoder = False
        print("Encoder unfrozen.")
    
    def forward(self, X):
        Z, mu, sigma = self.model.encoder(X)
        return self.clf(Z)

In [0]:
class DeepLSTM_VAE(nn.Module):
	def __init__(self, sequence_len, n_features, latent_dim, hidden_size=128, num_layers=2, batch_size=100, use_cuda=True):
		# ověřit predikci pro jiný batch size !!!!!!!!!!!!!!!!
		super(DeepLSTM_VAE, self).__init__()

		self.sequence_len = sequence_len
		self.n_features = n_features
		self.batch_size = batch_size
		self.hidden_size = hidden_size
		self.num_layers = num_layers
		self.use_cuda = use_cuda

		if self.use_cuda and torch.cuda.is_available():
			self.dtype = torch.cuda.FloatTensor
		else:
			self.dtype = torch.float32

		self.encoder_reshape = Reshape(out_shape=(self.sequence_len, self.n_features))
		self.encoder_lstm = nn.LSTM(
									input_size=n_features,
									hidden_size=hidden_size,
									num_layers=num_layers,
									batch_first=False,
									bidirectional=False
									) 
		self.encoder_output = VariationalLayer(
									in_features=hidden_size, 
									out_features=latent_dim, 
									return_KL=False
									)

		self.decoder_hidden = nn.Linear(
									in_features=latent_dim,
									out_features=hidden_size,
									bias=True	
									)
		self.decoder_lstm = nn.LSTM(
									input_size=1,
									hidden_size=hidden_size,
									num_layers=num_layers,
									batch_first=False,
									bidirectional=False
									) 
		self.decoder_output = RecurrentDecoderOutput(
									in_features=hidden_size,
									sequence_len=sequence_len,
									out_features=n_features,
									bias=True
									)
		self.decoder_input = torch.zeros(
									self.sequence_len, 
									self.batch_size, 
									self.n_features, 
									requires_grad=True
									).type(self.dtype)
		self.decoder_c_0 = torch.zeros(
									self.num_layers,
									self.batch_size,
									self.hidden_size,
									requires_grad=True 
									).type(self.dtype)


	def encoder(self, x_in):
		x = self.encoder_reshape(x_in)
		#set_trace()
		x = x.permute(1, 0, 2)
  
		_,(h_end, c_end) = self.encoder_lstm(x)
		h_end = h_end[-1, :, :] # shape(batch_size, num_features)
		return self.encoder_output(h_end)

	def decoder(self, z_in):
		h_state = self.decoder_hidden(z_in)
		#set_trace()
		h_0 = torch.stack([h_state for _ in range(self.num_layers)])
		lstm_output, _ = self.decoder_lstm(self.decoder_input, (h_0, self.decoder_c_0))
		mu, sigma = self.decoder_output(lstm_output)
		return mu, sigma

	def forward(self, x_in):
		z, mu, sigma = self.encoder(x_in)
		return self.decoder(z), mu, sigma

In [0]:
mod_l = DeepLSTM_VAE(
    sequence_len=160, 
    n_features=1, 
    latent_dim=15, 
    hidden_size=128, 
    num_layers=1, 
    batch_size=300, 
    use_cuda=True)

In [19]:
mod_l.cuda()

DeepLSTM_VAE(
  (encoder_reshape): Reshape()
  (encoder_lstm): LSTM(1, 128)
  (encoder_output): VariationalLayer(
    (mu): Linear(in_features=128, out_features=15, bias=True)
    (rho): Linear(in_features=128, out_features=15, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
  (decoder_hidden): Linear(in_features=15, out_features=128, bias=True)
  (decoder_lstm): LSTM(1, 128)
  (decoder_output): RecurrentDecoderOutput(
    (mu): Linear(in_features=128, out_features=1, bias=True)
    (rho): Linear(in_features=20480, out_features=1, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
)

In [20]:
mod_l.load_state_dict(torch.load("models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300.pt").state_dict())

<All keys matched successfully>

In [0]:
DNN1 = nn.Sequential(
            nn.Linear(in_features=15, out_features=128),
            nn.Sigmoid(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=128),
            nn.Sigmoid(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=4),
)

In [0]:
model = RNN_Encoder_Classifier(model=mod_l,clf= DNN1)

In [23]:
model.freeze_encoder()

Encoder frozen.


# training loop

In [0]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-2)

In [25]:
m1 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1),
        tensorboard=True,
        model_name="DNN_on_LSTM-VAE_sampling_latent_space_UNFREEZE_lr-{1e-2,1e-3}_epochs-512",
        device=torch.device("cpu"),
        verbose=True
        )

cuda:0


In [26]:
lh = m1(epochs=50, train_loader=train_loader, validation_loader=valid_loader)

/usr/local/lib/python3.6/dist-packages/torch/jit/__init__.py:1037: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[292, 1] (1.4314641952514648 vs. 1.5071825981140137) and 2046 other locations (99.95%)
  check_tolerance, _force_outplace, True, _module_class)
/usr/local/lib/python3.6/dist-packages/torch/jit/__init__.py:1037: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[78, 1] (1.3177536725997925 vs. 1.4737449884414673) and 2045 other locations (99.90%)
  check_tolerance, _force_outplace, True, _module_class)
/usr/local/lib/python3.6/dist-packages/torch/jit/__init__.py:1037: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolera

Epoch [1/50], average_loss:0.8494, validation_loss:0.6870, val_accuracy:0.7020
Epoch [2/50], average_loss:0.6575, validation_loss:0.5994, val_accuracy:0.7620
Epoch [3/50], average_loss:0.6085, validation_loss:0.5660, val_accuracy:0.7663
Epoch [4/50], average_loss:0.5818, validation_loss:0.5345, val_accuracy:0.7878
Epoch [5/50], average_loss:0.5636, validation_loss:0.5203, val_accuracy:0.7921
Epoch [6/50], average_loss:0.5498, validation_loss:0.5111, val_accuracy:0.7873
Epoch [7/50], average_loss:0.5407, validation_loss:0.4985, val_accuracy:0.7927
Epoch [8/50], average_loss:0.5335, validation_loss:0.4921, val_accuracy:0.8002
Epoch [9/50], average_loss:0.5325, validation_loss:0.4859, val_accuracy:0.7953
Epoch [10/50], average_loss:0.5301, validation_loss:0.4858, val_accuracy:0.7988
Epoch [11/50], average_loss:0.5265, validation_loss:0.4789, val_accuracy:0.7979
Epoch [12/50], average_loss:0.5191, validation_loss:0.4794, val_accuracy:0.8000
Epoch [13/50], average_loss:0.5174, validation_lo

In [27]:
model.unfreeze_encoder()

Encoder unfrozen.


In [0]:
optimizer= torch.optim.Adam([{"params":model.model.parameters(), "lr":1e-5}, {"params":model.clf.parameters()}], lr=1e-3)

In [0]:
m1.optimizer = optimizer

In [31]:
lh = m1(epochs=range(50,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [51/200], average_loss:0.4845, validation_loss:0.4440, val_accuracy:0.8109
Epoch [52/200], average_loss:0.4822, validation_loss:0.4452, val_accuracy:0.8114
Epoch [53/200], average_loss:0.4803, validation_loss:0.4441, val_accuracy:0.8089
Epoch [54/200], average_loss:0.4761, validation_loss:0.4425, val_accuracy:0.8134
Epoch [55/200], average_loss:0.4739, validation_loss:0.4406, val_accuracy:0.8127
Epoch [56/200], average_loss:0.4736, validation_loss:0.4373, val_accuracy:0.8151
Epoch [57/200], average_loss:0.4741, validation_loss:0.4375, val_accuracy:0.8138
Epoch [58/200], average_loss:0.4701, validation_loss:0.4392, val_accuracy:0.8118
Epoch [59/200], average_loss:0.4688, validation_loss:0.4340, val_accuracy:0.8163
Epoch [60/200], average_loss:0.4697, validation_loss:0.4357, val_accuracy:0.8150
Epoch [61/200], average_loss:0.4671, validation_loss:0.4370, val_accuracy:0.8163
Epoch [62/200], average_loss:0.4687, validation_loss:0.4368, val_accuracy:0.8127
Epoch [63/200], average_loss

In [0]:
with torch.no_grad():
    x_pred = np.argmax(m1.model(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)

In [33]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.7946867981823128

In [34]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8314412045889101

In [35]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2358, 1729,   32,  116],
       [ 354, 9949,   36,  106],
       [  20,   47,  397,   15],
       [ 187,  164,   15, 1211]])

In [36]:
torch.save(m1.model, "DNN_on_RVAE_sampling_latent_space_UNFREEZE_m.IV_lr-{1e-2,1e-3}_epochs-512.pt")
torch.save(m1.model.state_dict(), "DNN_on_RVAE_sampling_latent_space_UNFREEZE_m.IV_lr-{1e-2,1e-3}_epochs-512_state_dict.pt")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNN_Encoder_Classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type DeepLSTM_VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
